In [13]:
import cv2
import numpy as np
import os
import tensorflow as tf
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [15]:
Train_dir='e:/imgsign'
#Test_dir='f:/DATASETS/Test'
IMG_SIZE=200
LR=1e-3
MODEL_NAME="hand_gesture"
def CreateLabel(img_name):
    word_label=img_name.split('(')[0]
    if word_label=="A ":
        return np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    elif word_label=="B ":
        return np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    elif word_label=="C ":
        return np.array([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
    elif word_label=="U ":
        return np.array([0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
    elif word_label=="F ":
        return np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0])
    elif word_label=="G ":
        return np.array([0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0])
    elif word_label=="H ":
        return np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0])
    elif word_label=="I ":
        return np.array([0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0])
    elif word_label=="M ":
        return np.array([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0])
    elif word_label=="N ":
        return np.array([0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0])
    elif word_label=="O ":
        return np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0])
    elif word_label=="R ":
        return np.array([0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0])
    elif word_label=="V ":
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0])
    elif word_label=="W ":
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0])
    elif word_label=="Y ":
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0])
    elif word_label=="D ":
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
    

In [25]:
def CreateTrainData():
    Trainig_data=[]
    for img in tqdm(os.listdir(Train_dir)):
        path=os.path.join(Train_dir,img)
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img_data=cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
        img_data=cv2.GaussianBlur(img_data,(21,21),0)
        img_data=cv2.threshold(img_data,100,255,cv2.THRESH_BINARY)[1]
        #cv2.imshow('images',img_data)
        Trainig_data.append([np.array(img_data),CreateLabel(img)])
    shuffle(Trainig_data)
    np.save('traindata.npy',Trainig_data)
    return Trainig_data

In [26]:
def CreateTestData():
    Testing_data=[]
    for img in tqdm(os.listdir(Test_dir)):
        path=os.path.join(Test_dir,img)
        img_num=img.split('.')[-1]
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img_data=cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
        img_data=cv2.GaussianBlur(img_data,(21,21),0)
        img_data=cv2.threshold(img,100,255,cv2.THRESH_BINARY)[1]
        Testing_data.append([np.array(img_data),img_num])
    shuffle(Testing_data)
    np.save('testdata.npy',Testing_data)
    return Testing_data

In [27]:
train=CreateTrainData()

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:17<00:00,  4.03it/s]


In [28]:
train_data=train[:-32]
test_data=train[-31:]

In [29]:
x_train=np.array([i[0] for i in train_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train=[i[1] for i in train_data]
x_test=np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_test=[i[1] for i in test_data]

In [30]:
tf.reset_default_graph()
convnet=input_data(shape=[None,IMG_SIZE,IMG_SIZE,1],name='input')
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,128,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)

convnet=fully_connected(convnet,1024,activation='relu')
convnet=dropout(convnet,0.8)
convnet=fully_connected(convnet,16,activation='softmax')
convnet=regression(convnet,optimizer='adam',learning_rate=LR,loss='categorical_crossentropy',name='targets')
model=tflearn.DNN(convnet,tensorboard_dir='log',tensorboard_verbose=0)




In [32]:
model.fit({'input':x_train},{'targets':y_train},n_epoch=30,validation_set=({'input':x_test},{'targets':y_test}),
         snapshot_step=7,show_metric=True,run_id=MODEL_NAME)

---------------------------------
Run id: hand_gesture
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 29
Validation samples: 31
--
Training Step: 1  | time: 3.498s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 | val_loss: 2.97314 - val_acc: 0.0323 -- iter: 29/29
--
Training Step: 2  | total loss: 2.56611 | time: 3.042s
| Adam | epoch: 002 | loss: 2.56611 - acc: 0.1241 | val_loss: 3.71538 - val_acc: 0.0323 -- iter: 29/29
--
Training Step: 3  | total loss: 2.61830 | time: 3.038s
| Adam | epoch: 003 | loss: 2.61830 - acc: 0.0508 | val_loss: 3.19695 - val_acc: 0.1290 -- iter: 29/29
--
Training Step: 4  | total loss: 2.68522 | time: 3.026s
| Adam | epoch: 004 | loss: 2.68522 - acc: 0.0644 | val_loss: 2.95977 - val_acc: 0.0645 -- iter: 29/29
--
Training Step: 5  | total loss: 2.60534 | time: 3.012s
| Adam | epoch: 005 | loss: 2.60534 - acc: 0.0676 | val_loss: 2.96819 - val_acc

In [31]:
test_data

[[array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])],
 [array([[255, 255, 255, ...,   0,   0,   0],
         [255, 255, 255, ...,   0,   0,   0],
         [255, 255, 255, ...,

In [53]:
import cv2
img=cv2.imread("e:/X.png",0)
img=cv2.resize(img,(50,50))

In [51]:
model.predict()